<div style="border:3px solid black; padding:20px">
    

## * 아파트 실거래가 예측 경진대회

### + 배경
+ 국토부 실거래가 데이터를 가공한 유사 데이터로 진행되었으며, 아파트 실거래가를 예측하는 문제  


+ 현재는 교육용 대회로 전환됨
+ 대회 링크: https://dacon.io/competitions/open/235537/overview/

</div>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import requests
import json

import lightgbm as lgb

from sklearn import preprocessing
from sklearn.model_selection import KFold

### 특성별 데이터 개수
+ dong : 473
+ addr_kr : 12533
+ jibun : 8961
+ apartment_id : 12533
+ apt : 10440

## < Data load >

+ transaction_real_price는 min, max의 차이, std가 크므로 로그를 취해준다

In [ ]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

park= pd.read_csv('./data/park.csv')
day_care_center = pd.read_csv('./data/day_care_center.csv')
# park.gu 에서 추출하여 labeling 후 csv로 저장한 파일
gu = pd.read_csv('./data/gu.csv')

In [ ]:
print(train.transaction_real_price.describe())

train.transaction_real_price = np.log1p(train.transaction_real_price)

## < preprocessing and feature engineering >
### Center, Park.csv를 통한 특성 추출

+ 유치원 타입별로 차이가 있는줄 알았지만, 그렇지 않음
+ 결과에 영향을 거의 주지 않음
+ 공원은 '동'단위로 나누어져 있으며, 대체로 공원이 있는 동의 집값이 더 높음
+ 묘지공원을 포함한 동은 다른 동에 비해 집값이 낮음
  
  
+ 공원이 없는 동은 전부 0으로 대체

In [ ]:
gu_day_care_baby_num = day_care_center.groupby(['gu'])['day_care_baby_num'].agg({'mean'})

park = pd.merge(park, gu_day_care_baby_num, how = 'left', on = 'gu')
park.fillna(0, inplace = True)
park = park.rename(columns = {'mean':'day_care_baby_num_mean'})
ppark = park.loc[:, ['dong', 'park_type', 'day_care_baby_num_mean']] 
ppark = ppark[['dong']].join(pd.get_dummies(ppark['park_type'])).join(ppark['day_care_baby_num_mean']).groupby('dong').max()

train = pd.merge(train, ppark, how = 'left', on = 'dong')
test = pd.merge(test, ppark, how = 'left', on = 'dong')

train.fillna(0, inplace = True)
test.fillna(0, inplace = True)

+ 전용면적을 평 단위로 나누고, 평당 가격 특성을 만듬

In [ ]:
sns.lmplot('floor','transaction_real_price',data=train.sample(1000),order=2,scatter_kws={'alpha':0.1})

train['hangang']=train['dong'].isin(['성수동1가','삼성동','이촌동','공덕동','서교동','한강로3가','목동']).astype(int)
test['hangang']=test['dong'].isin(['성수동1가','삼성동','이촌동','공덕동','서교동','한강로3가','목동']).astype(int)

train['living'] = train['exclusive_use_area'] / 3.30578532
test['living'] = test['exclusive_use_area'] / 3.30578532

train['per_price'] = train['transaction_real_price'] / train['living']

+ 동단위로 아파트의 총 개수를 구함

In [ ]:
train=pd.merge(train,train.groupby(['dong']).size().reset_index(name='dong_count'),on=['dong'],how='left')
test=pd.merge(test,train.groupby(['dong']).size().reset_index(name='dong_count'),on=['dong'],how='left')

+ 동 내부에서도 지번별로 아파트 실거래가가 다를거라고 예상함

In [ ]:
dong_jibun_price = train.groupby(['dong', 'jibun'])['transaction_real_price'].agg({'mean'}).reset_index()

train = pd.merge(train, dong_jibun_price, how = 'left', on = ['dong', 'jibun'])
train = train.rename(columns = {'mean':'dong_jibun_price'})

test = pd.merge(test, dong_jibun_price, how = 'left', on = ['dong', 'jibun'])
test = test.rename(columns = {'mean':'dong_jibun_price'})
test.fillna(np.mean(test['dong_jibun_price']), inplace = True)

+ 동별로 실거래가 평균과 분산 특성 추가
+ 도시 및 지번별 실거래가 평균값 특성 추가
+ 도시 및 아파트별 실거래가 평균값 특성 추가

In [ ]:
# dong encoding
dong_price = train.groupby(['dong'])['per_price'].agg({'mean', 'var'}).reset_index()
train = pd.merge(train, dong_price, how = 'left', on = 'dong')
test = pd.merge(test, dong_price, how = 'left', on = 'dong')

for df in [train, test]:
    df['dong_mean'] = df['mean'] * df['living']
    df['dong_var'] = df['var'] * df['living']
    del df['mean'], df['var']
    
train = train.fillna(np.mean(train['dong_var']))

# addr_kr encoding
c_addr = train.groupby(['city', 'addr_kr'])['transaction_real_price'].agg({'mean'})
train = pd.merge(train, c_addr, how = 'left', on = ['city', 'addr_kr' ])
train = train.rename(columns = {'mean':'c_addr'})
test = pd.merge(test, c_addr, how = 'left', on = ['city', 'addr_kr'])
test = test.rename(columns = {'mean':'c_addr'})
test.fillna(np.mean(test['c_addr']), inplace = True)

# apartment_id encoding
c_a = train.groupby(['city', 'apartment_id'])['transaction_real_price'].agg({'mean'}).reset_index()
train = pd.merge(train, c_a, how = 'left', on = ['city', 'apartment_id'])
train = train.rename(columns = {'mean':'c_a'})
test = pd.merge(test, c_a, how = 'left', on = ['city', 'apartment_id'])
test = test.rename(columns = {'mean':'c_a'})
test.fillna(np.mean(test['c_a']), inplace = True)

+ 년, 월 추출

In [ ]:
train['year'] = train['transaction_year_month'].apply(lambda x : str(x)[:4])
train['month'] = train['transaction_year_month'].apply(lambda x : str(x)[4:])

test['year'] = test['transaction_year_month'].apply(lambda x : str(x)[:4])
test['month'] = test['transaction_year_month'].apply(lambda x : str(x)[4:])

train['year'] = train['year'].astype(np.int)
test['year'] = test['year'].astype(np.int)

train['month'] = train['month'].astype(np.int)
test['month'] = test['month'].astype(np.int)

+ transaction_date 특성을 LabelEncoder를 통해 labeling 해줌
+ 거래시점과 완성년도의 차이를 계산한 특성을 추가
+ 아파트가 완성된 시점이 오래될수록 실거래가가 낮을 것이라고 판단
+ 아파트가 30년이 지난 경우, 재건축되었을 확률이 높으므로 이를 특성으로 추가함

In [ ]:
# transaction_date labeling
date_label = preprocessing.LabelEncoder()
date_label.fit(train.transaction_date)
train['transaction_date'] = date_label.transform(train.transaction_date)
test['transaction_date'] = date_label.transform(test.transaction_date)

# 거래시점과 완성년도의 차이
train['transaction_diff'] = train['year'].astype(np.int) - train['year_of_completion']
test['transaction_diff'] = test['year'].astype(np.int) - test['year_of_completion']

train['is_rebuild']=(train['transaction_diff']>=30).astype(int)
test['is_rebuild']=(test['transaction_diff']>=30).astype(int)

+ 각 도시에서 거래시점과 완성년도의 차이름 기준으로 평균 특성을 추가함
+ 가장 최근에 지어진 아파트일수록 기존 데이터에 존재하지 않아 최솟값으로 대체함

In [ ]:
# 각 도시에서 완성년도와 거래시점과 완성년도의 차이를 기준으로
c_y_td = train.groupby(['city', 'year_of_completion', 'transaction_diff'])['transaction_real_price'].agg({'mean'}).reset_index()

train = pd.merge(train, c_y_td, how = 'left', on = ['city', 'year_of_completion', 'transaction_diff'])
train = train.rename(columns = {'mean':'c_y_td'})

test = pd.merge(test, c_y_td, how = 'left', on = ['city', 'year_of_completion', 'transaction_diff'])
test = test.rename(columns = {'mean':'c_y_td'})

# 주로 낮은 년도의 수가 채워지지 않으므로 평균이 아닌 min값으로 채워준다  
test.fillna(np.min(test['c_y_td']), inplace = True)

+ 각 도시에서 거래시점과 날짜를 기준으로 평균값 특성을 추가함
+ 가장 최근인 2017.12월이 없기때문에 최댓값으로 이를 대체함
+ 최근이기 때문에 실거래가의 평균이 높을 것으로 판단하였음

In [ ]:
# 각 도시에서 거래시점+날짜를 기준으로
c_tym_td = train.groupby(['city', 'transaction_year_month', 'transaction_date'])['transaction_real_price'].agg({'mean'}).reset_index()

train = pd.merge(train, c_tym_td, how = 'left', on = ['city', 'transaction_year_month', 'transaction_date'])
train = train.rename(columns = {'mean':'c_tym_td'})

test = pd.merge(test, c_tym_td, how = 'left', on = ['city', 'transaction_year_month', 'transaction_date'])
test = test.rename(columns = {'mean':'c_tym_td'})

# 가장 최근인 2017 12월이 없으므로 평균이 아닌 최댓값으로 채워준다 
test.fillna(np.max(test['c_tym_td']), inplace = True)

+ 서울과 부산을 따로 학습시키기 위해 이를 나누어줌

In [ ]:
city_label = {'서울특별시':0,'부산광역시':1}

train['city'] = train['city'].map(city_label)
test['city'] = test['city'].map(city_label)

seoul_transaction_id = test[test['city'] == 0]['transaction_id']
busan_transaction_id = test[test['city'] == 1]['transaction_id']

In [ ]:
# 에측에 사용되지 않을 열들 삭제
del train['transaction_id']; del test['transaction_id']
del train['apartment_id']; del test['apartment_id']
del train['dong']; del test['dong']
del train['apt']; del test['apt']
del train['addr_kr']; del test['addr_kr']
del train['jibun']; del test['jibun']
del train['living']; del test['living']
del train['transaction_year_month']; del test['transaction_year_month']
del train['transaction_diff']; del test['transaction_diff']
del train['transaction_date']; del test['transaction_date']
del train['per_price']

## < Training >

+ xgb로도 실험해보았지만, lgb의 경우가 더 좋은 성능을 보여주었음

In [ ]:
seoul_train = train[train['city'] == 0].reset_index(drop = True)
seoul_test = test[test['city'] == 0].reset_index(drop = True)

busan_train = train[train['city'] == 1].reset_index(drop = True)
busan_test = test[test['city'] == 1].reset_index(drop = True)

del seoul_train['city']; del seoul_test['city']
del busan_train['city']; del busan_test['city']

folds = KFold(n_splits=5,shuffle=True,random_state=15)

param = {'num_leaves': 100,
         'min_data_in_leaf': 15, 
         'objective':'regression',
         'max_depth': 6,
         'learning_rate': 0.1,
         "min_child_samples": 30,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1}

train_columns = train.drop(['city', 'transaction_real_price'], axis = 1).columns

## < 부산 학습 >

In [ ]:
busan_predictions_ops=np.zeros(len(busan_test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(busan_train.values, busan_train['transaction_real_price'].values)):
    trn_data = lgb.Dataset(busan_train[train_columns].iloc[trn_idx],label=busan_train['transaction_real_price'].iloc[trn_idx])#,weight=game_by_game.loc[(game_by_game['year']<year)&(game_by_game['AB']>0)]['AB'].iloc[trn_idx])
    val_data = lgb.Dataset(busan_train[train_columns].iloc[val_idx],label=busan_train['transaction_real_price'].iloc[val_idx])#,weight=game_by_game.loc[(game_by_game['year']<year)&(game_by_game['AB']>0)]['AB'].iloc[val_idx])

    num_round = 10000
    clf = lgb.train(param,
                    trn_data,
                    num_round,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=1000,
                    early_stopping_rounds = 200)


    busan_predictions_ops += clf.predict(busan_test[train_columns], num_iteration=clf.best_iteration)

busan_predictions_ops/=5

In [ ]:
busan_sub = pd.DataFrame({'transaction_id':busan_transaction_id, 
                          'transaction_real_price':busan_predictions_ops})

## < 서울 학습 >

In [ ]:
seoul_predictions_ops=np.zeros(len(seoul_test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(seoul_train.values, seoul_train['transaction_real_price'].values)):
    trn_data = lgb.Dataset(seoul_train[train_columns].iloc[trn_idx],label=seoul_train['transaction_real_price'].iloc[trn_idx])#,weight=game_by_game.loc[(game_by_game['year']<year)&(game_by_game['AB']>0)]['AB'].iloc[trn_idx])
    val_data = lgb.Dataset(seoul_train[train_columns].iloc[val_idx],label=seoul_train['transaction_real_price'].iloc[val_idx])#,weight=game_by_game.loc[(game_by_game['year']<year)&(game_by_game['AB']>0)]['AB'].iloc[val_idx])

    num_round = 10000
    clf = lgb.train(param,
                    trn_data,
                    num_round,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=1000,
                    early_stopping_rounds = 200)


    seoul_predictions_ops += clf.predict(seoul_test[train_columns], num_iteration=clf.best_iteration)

seoul_predictions_ops/=5

In [ ]:
seoul_sub = pd.DataFrame({'transaction_id':seoul_transaction_id, 
                          'transaction_real_price':seoul_predictions_ops})

## < 제출 형식 > 

In [ ]:
concat_df = pd.concat((busan_sub, seoul_sub), axis = 0)

id = pd.read_csv('./data/submission.csv')
id = id.loc[:, 'transaction_id']

sub = pd.merge(id, concat_df, how = 'left', on = 'transaction_id')
sub['transaction_real_price'] = np.expm1(sub['transaction_real_price'])

sub.to_csv('./sub/my_submission.csv', index = False)